# Car ride-share potential in mid-size U.S. cities from geographic spread

## (notebook 3: cCITY_INDEX=16)

Third notebook for the IBM Data Science Specialization on Coursera, which contains the consolidated methods and functions from the first two notebooks, to serve as a template to be used per-city.



## Installs (if needed) and imports

See notesbooks 1 and 2 for details.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import math
import json

try:
    from geopy.geocoders import Nominatim
    print(Nominatim)
except:
    !conda install -c conda-forge geopy=1.18.1 --yes
    
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

try:
    import folium
    print(folium)
except:
    !conda install -c conda-forge folium=0.8.0 --yes
    
import folium
import time
    
try:
    import pandas_profiling
    print(pandas_profiling)
except:
    !conda install -c conda-forge pandas-profiling=1.4.1 --yes
    
import pandas_profiling

<class 'geopy.geocoders.osm.Nominatim'>
<module 'folium' from '/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/folium/__init__.py'>
<module 'pandas_profiling' from '/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas_profiling/__init__.py'>


## Foursquare secret

In [2]:
# (paste secret from local file)


In [3]:
print('CLIENT_ID set: {}'.format(CLIENT_ID is not None))
print('CLIENT_SECRET set: {}'.format(CLIENT_SECRET is not None))

VERSION = '20180605' # Foursquare API version

CLIENT_ID set: True
CLIENT_SECRET set: True


## Which city?

In [4]:
CITY_INDEX=16
RADIUS=800
MAX_COORDS=1000
MAP_ZOOM_LEVEL=11
MARKER_ZOOM=0.1

## All function definitions

See notebooks 1 and 2 for details.

In [5]:
print('Starting: Get list of cities')

url = 'https://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=883568308'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url, 'lxml')
city_table = soup.find('table', { 'class' : 'wikitable sortable' })

print('---------CITIES TABLE (raw)')
print("{}\n\n   [...]\n\n{}".format(str(city_table)[:500].replace('\n', '').replace('<tr>', '\n\n<tr>'), str(city_table)[-500:]))


l = []

table_rows = city_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("(ignoring empty row)")
        test_size = 0
    else:
        test_size = int(row[3].replace(',', ''))
        
    if test_size >= 300000 and test_size <= 400000:
        city_name = re.sub('\[.*\]', '', row[1])
        city_state = row[2]
        city_estd_pop2017 = test_size
        city_latlongraw = re.sub('^.*/', '', re.sub('\(.*\)', '', row[10])).replace(' ', '')
        # strip non-ASCII residue
        city_latlongraw = city_latlongraw.encode('ascii',errors='ignore').decode()
        city_lat = float(re.sub(';.*$', '', city_latlongraw))
        city_long = float(re.sub('^.*;', '', city_latlongraw))
        l.append([city_name, city_state, city_estd_pop2017, city_lat, city_long])

cities_df = pd.DataFrame(l)
cities_df.columns = ['City name', 'City state', 'Population', 'Latitude', 'Longitude']

print('----------CITIES DATAFRAME')
print(cities_df)


def getVenuesNearLatLong(latitude, longitude, radius=500, limit=100, verbose=True):
    
    venues_list=[]
                
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            latitude, 
            longitude, 
            radius, 
            limit)
            
    # make the GET request (on error try four more times before giving up)
    num_tries = 0
    results = [] # assume no venues if persistent error
    
    while num_tries < 5:
        num_tries +=1
        try:
            results_raw = requests.get(url)
            results = results_raw.json()["response"]['groups'][0]['items']
        except:
            print('(err)', end='')
            time.sleep(2) # sleep for two seconds, then retry
        
    # return only relevant information for each nearby venue
    venues_list.append([(
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    if (len(results) > 0):
        nearby_venues.columns = [
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Category']
    
    if verbose:
        print('found {} venues within {} meters of {}/{}'.format(len(results), radius, latitude, longitude))
    else:
        print('{}'.format(len(results)), end='. ')
    
    return(nearby_venues)


def get_venues_in_hex_grid(latitude, longitude, venues_dict, this_coord, radius=500, limit=100, new_coords=[], verbose=True):
    '''
    Calls Foursquare in a hex grid around a given coordinate point. If venues
    have already been searched on one of the hex grid points, that result is
    kept and no new search is executed.
    
    Parameters:
    
    latitude and longitude are as of the origin coordinate (0, 0),
    venues_dict are the venues found so far (dictionary keys are a coordinate tuple),
    this_coord is the center coordinate around which the hex grid is to be searched,
    radius is the radius [meters] to search around a coordinate point,
    limit is the maximum number of venues to return from a Foursquare search.
    new_coords is a list of coordinate points that wasn't probed yet
    
    Returns a list of new coordinate tuples appended to the new_coords parameter, if any
    '''
    
    r_earth = 6378000. # approximate radius of the Earth in meters
    pi = math.pi
    sqrt_three = math.sqrt(3.)
    overlap = 1.4 # 40% overlap
    
    cx = this_coord[0] # center X
    cy = this_coord[1] # center Y
    hex_coords = [ (cx-1,cy), (cx+1, cy), (cx,cy-1), (cx,cy+1), (cx-1,cy+1), (cx+1,cy-1) ] # the gex grid around this_coord
    
    if (cx, cy) in new_coords:
        new_coords.remove((cx, cy))
    
    for this_hex in hex_coords:
        if not this_hex in venues_dict:
            # the coordinate has not been searched for
            
            # get the x- and y-step from a hex grid; start with a square grid (letting the circles overlap a bit):
            dx_square = this_hex[0] * radius * ( overlap / 2. )
            dy_square = this_hex[1] * radius * ( overlap / 2. )
            # now convert to a hex grid:
            dx = dx_square + dy_square / 2.
            dy = dy_square * ( sqrt_three / 2. )
            # approximate the center point's latitude and longitude assuming locally flat Earth
            hex_latitude  = latitude  + (dy / r_earth) * (180 / pi);
            hex_longitude = longitude + (dx / r_earth) * (180 / pi) / math.cos(latitude * pi/180);
            
            if verbose:
                print('getting coordinate {}...'.format(this_hex))
            else:
                print('({},{}):'.format(this_hex[0], this_hex[1]), end='')
                
            this_venues = getVenuesNearLatLong(hex_latitude, hex_longitude, radius=radius, limit=limit, verbose=verbose)
            venues_dict[this_hex] = this_venues
            if not this_hex in new_coords:
                new_coords.append(this_hex)
    
    return new_coords


def make_map_from_dict(lat_orig, long_orig, venues_dict, zoom_start,
                       city_name='(city_name)', city_state='(city_state)',
                       radius_zoom=1.0, width=600, height=600,
                       no_touch=True,
                       zoom_control=False,
                       failsafe_abort_num=4000,
                       simple_rendering=False):
    
    if zoom_control == False:
        min_zoom = zoom_start
        max_zoom = zoom_start
    else:
        min_zoom = 0
        max_zoom = 16
    
    new_map = folium.Map(
        location=[lat_orig, long_orig],
        zoom_start=zoom_start,
        width=width,
        height=height,
        control_scale=True,
        no_touch=no_touch,
        min_zoom=min_zoom,
        max_zoom=max_zoom
    )
    
    num_markers=0

    # add markers to map
    for coords, venues in venues_dict.items():
        num_markers += 1
        if (venues.shape[0] > 0) and (num_markers < failsafe_abort_num): 
            if simple_rendering:
                folium.Circle(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    color=None,
                    fill=True,
                    fill_color='#005090',
                    fill_opacity=0.7).add_to(new_map)
            else:
                label = '{}, # venues: {}'.format(coords, venues.shape[0])
                label = folium.Popup(label, parse_html=True)
                folium.CircleMarker(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    popup=label,
                    color='blue',
                    fill=True,
                    fill_color='#3186cc',
                    fill_opacity=0.7,
                    parse_html=False).add_to(new_map)
        
    legend_html = ('<div style="position: fixed; top: 30px; left: 50px; width: 450px;' 
                + 'height: 30px; border: 2px solid grey; z-index: 9999; font-size: 16px; background-color: white">' 
                + '&nbsp;{},&nbsp;{}' 
                + '</div>').format(
                     city_name,
                     city_state
                     )
    new_map.get_root().html.add_child(folium.Element(legend_html))
     
    return new_map


def find_venues_geo_distribution(cities_df, city_index, max_coords_tested=100, radius=1500, limit=100, verbose=True):
    
    city_name = cities_df['City name'][city_index]
    city_state = cities_df['City state'][city_index]
    
    # initialize venues_dict with the venues dataframe at the origin
    venues_dict = {}
    origin_coord = (0,0)
    lat_orig = cities_df.Latitude[city_index]
    long_orig = cities_df.Longitude[city_index]
    print('[test #1 of {}] (0,0):'.format(max_coords_tested), end='')
    venues_df = getVenuesNearLatLong(lat_orig, long_orig, radius=radius, verbose=verbose)
    venues_dict[origin_coord] = venues_df
    
    # mark the origin as the first (and only) coordinate point not yet explored
    new_coords = [(0, 0)]
    num_coords_tested = 1
    
    while num_coords_tested < max_coords_tested:
        
        highest_venues = -1
        new_test_coord = None
        
        for this_coord in new_coords:
            venues_df = venues_dict[this_coord]
            if venues_df.shape[0] > highest_venues:
                new_test_coord = this_coord
                highest_venues = venues_df.shape[0]
        
        # call the hex grid exploration function
        num_coords_tested += 1
        print('[test #{} of {}]'.format(num_coords_tested, max_coords_tested), end=' ')
        new_coords = get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, new_test_coord, radius=radius, new_coords=new_coords, limit=limit, verbose=verbose )
        
    return lat_orig, long_orig, venues_dict, city_name, city_state


def is_indicator_venue(venue_type):
    if venue_type is None:
        return False
    
    magic_words = [
        'yoga',
        'salad',
        'coworking',
        'alternative',
        'bike',
        'fitness',
        'running',
        'jogging',
        'cycling',
        'cycle',
        'athletics',
        'gluten',
        'health',
        'recreation',
        'tennis',
        'vegetarian',
        'vegan',
        'tennis',
        'disc golf',
        'pilates',
        'share',
        'sharing',
        'incubat',
        'innovat'
    ]
    return any(substring in venue_type.lower() for substring in magic_words)

print('Perform some tests on indicator venues:')
print(is_indicator_venue(None))
print(is_indicator_venue(''))
print(is_indicator_venue('Salad Bar'))
print(is_indicator_venue('Chinese Restaurant'))
print(is_indicator_venue('Coworking space'))


def aggregate_venues_dict(venues_dict):
    venues_agg = []
    venues_types = []
    for coord, venues in venues_dict.items():
        num_venues = venues.shape[0]
        num_indicators = 0
        if num_venues > 0:
            for this_type in venues['Venue Category'].values:
                venues_types.append(this_type)
                if is_indicator_venue(this_type):
                    num_indicators += 1
        venues_agg.append([coord, num_venues, num_indicators])
    venues_agg_df = pd.DataFrame(venues_agg)
    venues_agg_df.columns = ['coord', 'num_venues', 'num_indicators']
    return venues_agg_df, set(venues_types)


def find_venues_distance_to_center(venues_agg_df):
    '''
    Appends or updates x/y and distance-to-center columns of venues_agg_df.
    Also returns some aggregates that were calculated along the way.
    '''
    
    x, y = zip(*venues_agg_df['coord'])
    total_num_venues = venues_agg_df['num_venues'].sum()
    total_num_indicators = venues_agg_df['num_indicators'].sum()
    center_x = ( x * venues_agg_df['num_venues'] ).sum() / total_num_venues
    center_y = ( y * venues_agg_df['num_venues'] ).sum() / total_num_venues
    dx = x - center_x
    dy = y - center_y
    dist2 = dx * dx + dy * dy
    venues_agg_df['dist'] = np.sqrt(dist2)
    
    return center_x, center_y, total_num_venues, total_num_indicators


def gather_distance_distribution_stats(venues_agg_df, city_index=-1, city_display='(city name, state)', return_as_list_append_string=False):
    
    center_x, center_y, total_num_venues, total_num_indicators = find_venues_distance_to_center(venues_agg_df)
    num_venues_per_coord = venues_agg_df['num_venues']
    dist_per_coord = venues_agg_df['dist']
    
    # blow up the dist_per_coord series by the nunmber of venues at the coordinate
    
    dist_list = []
    for this_coord in zip(num_venues_per_coord, dist_per_coord):
        this_list = [float(this_coord[1])] * int(this_coord[0])
        dist_list.extend(this_list)
    
    dist_list_df = pd.DataFrame(dist_list)
    dist = dist_list_df[0]
    
    # now perform statistics on that new list
    
    mean = dist.mean()
    std = dist.std()
    p25 = dist.quantile(0.25)
    median = dist.quantile(0.5)
    p75 = dist.quantile(0.75)
    max_d = dist.max()
    iqr = p75 - p25
    kurt = dist.kurtosis() # "peaky-ness"
    mad = dist.mad() # mean absolute deviation
    skew = dist.skew() # skewedness
    
    if return_as_list_append_string:
        retS = '# Create list at the beginning:\n'
        retS += '#     l = []\n\n'
        retS += 'l.append(['
        retS += '{}, "{}", ({:.2f}, {:.2f}), '.format(city_index, city_display, center_x, center_y)
        retS += '{}, {}, {}, {}, '.format(total_num_venues, total_num_indicators, mean, std)
        retS += '{}, {}, {}, {}, '.format(p25, median, p75, max_d)
        retS += '{}, {}, {}, {}'.format(iqr, kurt, mad, skew)
        retS += '])\n\n'
        retS += '# Convert list to pandas DataFrame:\n'
        retS += '#     cities_stats_df = pd.DataFrame(l)\n'
        retS += '#     cities_stats_df.columns = ['
        retS += '"city index", "city name, state", "center coord", "total num venues", "total num indicators", '
        retS += '"mean distance to center", "std dev", "25th percentile", "median", "75th percentile", "max distance to center", '
        retS += '"interquartile range", "kurtosis", "mean absolute deviation", "skewedness"'
        retS += ']\n'
        return retS

    else:
        return center_x, center_y, total_num_venues, total_num_indicators, \
               mean, std, p25, median, p75, max_d,                         \
               iqr, kurt, mad, skew
    

    



Starting: Get list of cities
---------CITIES TABLE (raw)
<table class="wikitable sortable" style="text-align:center"><tbody>

<tr><th>2017<br/>rank</th><th>City</th><th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup></th><th>2017<br/>estimate</th><th>2010<br/>Census</th><th>Change</th><th colspan="2">2016 land area</th><th colspan="2">2016 population density</th><th>Location</th></tr>

<tr><td>1</td><td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York 

   [...]

"latitude">38°21′14″N</span> <span class="longitude">121°58′22″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">38.3539°N 121.9728°W</span><span style="display:none">﻿ / <span class="geo">38.3539; -121.9728</span></span><span style="display:none">﻿ (<span class="fn org">Vacaville</span>)</span></span></s

## Quick test

In [6]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=CITY_INDEX, radius=RADIUS, max_coords_tested=10, verbose=False)

print("{}, {}\n".format(city_name, city_state))

venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)

print("\n{}\n".format(venues_agg_df.head(10)))

center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew \
    = gather_distance_distribution_stats(venues_agg_df)

print(center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew)

list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=CITY_INDEX,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nlist_append_string:\n\n{}'.format(list_append_string))

make_map_from_dict(
    lat_orig, long_orig, venues_dict, 13, city_name, city_state, radius_zoom=7,
    width=600, height=600, simple_rendering=True)


[test #1 of 10] (0,0):0. [test #2 of 10] (-1,0):3. (1,0):2. (0,-1):3. (0,1):1. (-1,1):1. (1,-1):0. [test #3 of 10] (-2,0):6. (-1,-1):3. (-2,1):7. [test #4 of 10] (-3,1):21. (-2,2):13. (-3,2):27. [test #5 of 10] (-4,2):25. (-3,3):31. (-4,3):32. [test #6 of 10] (-5,3):30. (-4,4):35. (-5,4):24. [test #7 of 10] (-3,4):26. (-4,5):27. (-5,5):27. [test #8 of 10] (-2,3):7. [test #9 of 10] (-6,3):23. (-5,2):27. (-6,4):24. [test #10 of 10] (-3,5):19. (-4,6):34. (-5,6):33. Henderson, Nevada


     coord  num_venues  num_indicators
0  (-4, 5)          27               1
1  (-5, 2)          27               2
2   (0, 0)           0               0
3  (-1, 0)           3               0
4  (-6, 3)          23               0
5  (-2, 2)          13               0
6  (-5, 5)          27               1
7  (-2, 1)           7               0
8  (-4, 2)          25               2
9  (-5, 6)          33               1

-3.99608610568 3.48336594912 511 21 1.7727817294149946 0.9368835981891133 1.1181576

## The real deal

In [7]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=CITY_INDEX, radius=RADIUS, max_coords_tested=MAX_COORDS, verbose=False)

print("\n\n{}, {}\n".format(city_name, city_state))
venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)
print('Total shape: {}\n\nSample 10 venues per coordinate (unsorted):'.format(venues_agg_df.shape))
print('{}\n'.format(venues_agg_df.head(10)))

list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=CITY_INDEX,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nResulting list append string for this city:\n\n{}'.format(list_append_string))


[test #1 of 1000] (0,0):0. [test #2 of 1000] (-1,0):3. (1,0):2. (0,-1):3. (0,1):1. (-1,1):1. (1,-1):0. [test #3 of 1000] (-2,0):6. (-1,-1):3. (-2,1):7. [test #4 of 1000] (-3,1):21. (-2,2):13. (-3,2):27. [test #5 of 1000] (-4,2):25. (-3,3):31. (-4,3):32. [test #6 of 1000] (-5,3):30. (-4,4):35. (-5,4):24. [test #7 of 1000] (-3,4):26. (-4,5):27. (-5,5):27. [test #8 of 1000] (-2,3):7. [test #9 of 1000] (-6,3):23. (-5,2):27. (-6,4):24. [test #10 of 1000] (-3,5):19. (-4,6):34. (-5,6):33. [test #11 of 1000] (-3,6):16. (-4,7):20. (-5,7):34. [test #12 of 1000] (-6,7):37. (-5,8):28. (-6,8):28. [test #13 of 1000] (-7,7):6. (-6,6):23. (-7,8):27. [test #14 of 1000] [test #15 of 1000] (-4,8):10. (-5,9):7. (-6,9):29. [test #16 of 1000] (-7,9):15. (-6,10):50. (-7,10):51. [test #17 of 1000] (-8,10):(err)26. (-7,11):100. (-8,11):67. [test #18 of 1000] (-6,11):100. (-7,12):100. (-8,12):100. [test #19 of 1000] (-5,11):43. (-6,12):74. (-5,10):29. [test #20 of 1000] (-7,13):80. (-8,13):81. [test #21 of 1000

100. [test #191 of 1000] (-37,28):39. [test #192 of 1000] (-38,26):100. (-38,27):53. [test #193 of 1000] (-34,19):12. (-33,18):35. [test #194 of 1000] (-35,29):65. (-36,29):80. [test #195 of 1000] (-37,29):37. [test #196 of 1000] (-39,26):74. (-38,25):100. (-39,27):69. [test #197 of 1000] (-39,25):70. (-38,24):55. (-37,24):61. [test #198 of 1000] (-32,17):28. (-31,17):37. [test #199 of 1000] (-31,25):56. [test #200 of 1000] (-33,28):45. (-34,29):84. [test #201 of 1000] (-30,17):18. [test #202 of 1000] (-30,21):50. [test #203 of 1000] (-30,25):59. [test #204 of 1000] (-33,29):51. (-34,30):78. (-35,30):88. [test #205 of 1000] (-36,30):56. (-35,31):79. (-36,31):93. [test #206 of 1000] (-37,31):54. (-36,32):96. (-37,32):31. [test #207 of 1000] (-35,32):97. (-36,33):97. (-37,33):21. [test #208 of 1000] (-34,32):67. (-35,33):79. (-34,31):100. [test #209 of 1000] (-33,31):48. (-33,30):60. [test #210 of 1000] (-36,34):94. (-37,34):69. [test #211 of 1000] (-35,34):69. (-36,35):100. (-37,35):92.

25. (-32,36):13. (-31,34):27. [test #405 of 1000] [test #406 of 1000] (-44,22):18. (-43,22):3. [test #407 of 1000] (-32,16):19. [test #408 of 1000] [test #409 of 1000] (-35,19):16. [test #410 of 1000] (-47,35):21. (-47,36):12. [test #411 of 1000] (-48,29):13. (-49,29):1. [test #412 of 1000] [test #413 of 1000] [test #414 of 1000] [test #415 of 1000] (-30,34):18. (-31,33):16. (-30,33):19. [test #416 of 1000] [test #417 of 1000] (-19,15):27. (-19,14):21. [test #418 of 1000] (-18,15):29. (-19,16):18. (-18,14):17. [test #419 of 1000] (-17,15):26. (-18,16):16. (-17,14):18. [test #420 of 1000] [test #421 of 1000] [test #422 of 1000] [test #423 of 1000] [test #424 of 1000] (-32,33):19. [test #425 of 1000] [test #426 of 1000] [test #427 of 1000] (-43,21):18. [test #428 of 1000] (-41,33):9. [test #429 of 1000] (-42,35):9. [test #430 of 1000] (-48,38):14. (-48,39):10. [test #431 of 1000] (-47,34):20. [test #432 of 1000] (-40,34):19. [test #433 of 1000] (-16,15):32. (-17,16):5. (-16,14):24. [test

23. (-54,37):14. (-55,39):28. [test #619 of 1000] (-56,39):25. (-55,40):20. (-56,40):29. [test #620 of 1000] (-57,40):18. (-56,41):28. (-57,41):25. [test #621 of 1000] (-55,41):27. (-56,42):91. (-57,42):66. [test #622 of 1000] (-55,42):55. (-56,43):82. (-57,43):77. [test #623 of 1000] (-55,43):86. (-56,44):72. (-57,44):56. [test #624 of 1000] (-54,43):57. (-55,44):54. (-54,42):62. [test #625 of 1000] (-58,43):56. (-58,44):49. [test #626 of 1000] (-56,45):27. (-57,45):13. [test #627 of 1000] (-58,42):44. [test #628 of 1000] (-53,42):28. (-54,41):12. (-53,41):4. [test #629 of 1000] (-53,43):31. (-54,44):44. [test #630 of 1000] (-58,45):7. [test #631 of 1000] (-59,43):12. (-59,44):26. [test #632 of 1000] [test #633 of 1000] (-55,45):15. [test #634 of 1000] (-59,45):12. [test #635 of 1000] (-59,42):4. (-58,41):4. [test #636 of 1000] (-53,44):27. (-54,45):5. [test #637 of 1000] (-52,43):22. (-52,42):28. [test #638 of 1000] (-52,41):12. [test #639 of 1000] (-51,42):19. (-51,41):10. [test #64

11. [test #823 of 1000] [test #824 of 1000] [test #825 of 1000] (-44,21):19. [test #826 of 1000] (-45,21):20. (-44,20):20. (-43,20):21. [test #827 of 1000] (-43,19):26. (-42,19):13. [test #828 of 1000] (-44,19):11. (-43,18):23. (-42,18):20. [test #829 of 1000] (-44,18):4. (-43,17):14. (-42,17):30. [test #830 of 1000] (-41,17):22. (-42,16):23. (-41,16):29. [test #831 of 1000] (-40,16):9. (-41,15):27. (-40,15):34. [test #832 of 1000] (-39,15):7. (-40,14):37. (-39,14):11. [test #833 of 1000] (-41,14):39. (-40,13):70. (-39,13):37. [test #834 of 1000] (-41,13):56. (-40,12):77. (-39,12):63. [test #835 of 1000] (-41,12):40. (-40,11):58. (-39,11):79. [test #836 of 1000] (-38,11):56. (-39,10):58. (-38,10):59. [test #837 of 1000] (-38,12):33. [test #838 of 1000] (-37,10):24. (-38,9):61. (-37,9):50. [test #839 of 1000] (-39,9):33. (-38,8):46. (-37,8):40. [test #840 of 1000] (-41,11):25. (-40,10):19. [test #841 of 1000] [test #842 of 1000] (-42,13):34. (-42,14):21. [test #843 of 1000] (-37,11):16.

In [8]:
make_map_from_dict(
    lat_orig, long_orig, venues_dict, 11, city_name, city_state, radius_zoom=7,
    width=600, height=600, simple_rendering=True)
